In [9]:
import amino_acid
import numpy
import pandas as pd
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Activation 
from sklearn.model_selection import KFold
from IPython.display import display, HTML
import numpy as np

In [7]:
properties = pd.read_csv('gfp_properties.csv')
properties.head()

,0,1,2,3,4,5,6,7,8,9,...,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427
0,0.5,0.5,1.0,0.5,0.0,0.0,0.5,0.5,0.5,0.5,...,1.0,0.422222,0.144444,0.455556,0.711111,0.111111,0.111111,0.922222,0.355556,0.066667
1,0.5,0.5,1.0,0.5,0.0,0.0,0.5,0.5,0.5,0.5,...,1.0,0.422222,0.144444,0.455556,0.711111,0.111111,0.111111,0.922222,0.355556,0.066667
2,0.5,0.5,1.0,0.5,0.0,0.0,0.5,0.5,0.5,0.5,...,1.0,0.422222,0.144444,0.455556,0.711111,0.111111,0.111111,0.922222,0.355556,0.066667
3,0.5,0.5,1.0,0.5,0.0,0.0,0.5,0.5,0.5,0.5,...,1.0,0.422222,0.144444,0.455556,0.711111,0.111111,0.111111,0.922222,0.355556,0.066667
4,0.5,0.5,1.0,0.5,0.0,0.0,0.5,0.5,0.5,0.5,...,1.0,0.422222,0.144444,0.455556,0.711111,0.111111,0.111111,0.922222,0.355556,0.066667


In [6]:
labels = pd.read_csv('labels_float_brightness.csv')
labels = labels.drop(['SD of brightness'], axis = 1)
labels.head()

,Brightness
0,1.301030
1,3.325116
2,3.172078
3,3.641499
4,1.579974


In [24]:
# this gets the one hot enconded sequences and for each sequence concatenas teh amino acids (since each aa is a onehot list of len 20)
# the final product has dimensions of 51714 proteins by 238aa*20types =  (51714, 4760)
Y = np.array(labels['Brightness'])
X = np.array(properties)
np.shape(X)

(51714, 1428)

In [27]:
# simple split into train and test set, even == test, odd == train
nproteins = 51714
train_indices = np.array(range(1,nproteins,2)) 
test_indices = np.array(range(0,nproteins,2))
print("TRAIN INDEX:", train_indices,'\n', "TEST INDEX:", test_indices)

TRAIN INDEX: [    1     3     5 ... 51709 51711 51713] 
 TEST INDEX: [    0     2     4 ... 51708 51710 51712]


In [30]:
# Build the models for each taxonomy rank level
BATCH_SIZE = 20000 # batchsize = half of the data
EPOCHS = 4
STEPS_PER_EPOCH = 5 # len(train_indices) // BATCH_SIZE

# number of cell types (labels) in taxonomy rank
output_dim = 1
#number of genes
input_dim = np.shape(X)[1]
model = Sequential() 
model.add(Dense(256, input_dim=input_dim, activation='relu')) 

model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='relu'))

#Compile the model
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['accuracy']) 

In [31]:
print ('HALF-HALF DATA SPLIT FOR TRAINING AND VALIDATION OF LOGREG CLASSIFIER')
history = model.fit(
        x = np.array(X)[train_indices],
        y = np.array(Y)[train_indices],
        validation_data=(X[test_indices], Y[test_indices]),
        validation_steps=5,
        epochs=20,
        steps_per_epoch= 30)


HALF-HALF DATA SPLIT FOR TRAINING AND VALIDATION OF LOGREG CLASSIFIER
Train on 25857 samples, validate on 25857 samples
Epoch 1/20
30/30 [==============================] - 7s 243ms/step - loss: 1.5018 - acc: 0.0000e+00 - val_loss: 1.1119 - val_acc: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 7s 217ms/step - loss: 1.0830 - acc: 0.0000e+00 - val_loss: 1.0541 - val_acc: 0.0000e+00
Epoch 3/20
30/30 [==============================] - 6s 208ms/step - loss: 0.9916 - acc: 0.0000e+00 - val_loss: 0.9367 - val_acc: 0.0000e+00
Epoch 4/20
30/30 [==============================] - 6s 206ms/step - loss: 0.7953 - acc: 0.0000e+00 - val_loss: 0.7894 - val_acc: 0.0000e+00
Epoch 5/20
30/30 [==============================] - 7s 236ms/step - loss: 0.6726 - acc: 0.0000e+00 - val_loss: 0.6501 - val_acc: 0.0000e+00
Epoch 6/20
30/30 [==============================] - 7s 220ms/step - loss: 0.7004 - acc: 0.0000e+00 - val_loss: 0.6520 - val_acc: 0.0000e+00
Epoch 7/20
30/30 [======================

In [ ]:
print ('HALF-HALF DATA SPLIT FOR TRAINING AND VALIDATION OF LOGREG CLASSIFIER')
history = model.fit(
        x = np.array(X)[train_indices],
        y = np.array(Y)[train_indices],
        validation_data=(X[test_indices], Y[test_indices]),
        validation_steps=5,
        epochs=20,
        steps_per_epoch= 30)

HALF-HALF DATA SPLIT FOR TRAINING AND VALIDATION OF LOGREG CLASSIFIER
Train on 25857 samples, validate on 25857 samples
Epoch 1/20
30/30 [==============================] - 7s 220ms/step - loss: 0.3910 - acc: 0.0000e+00 - val_loss: 0.3772 - val_acc: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 6s 211ms/step - loss: 0.4450 - acc: 0.0000e+00 - val_loss: 0.4049 - val_acc: 0.0000e+00
Epoch 3/20
30/30 [==============================] - 7s 236ms/step - loss: 0.4176 - acc: 0.0000e+00 - val_loss: 0.8204 - val_acc: 0.0000e+00
Epoch 4/20
30/30 [==============================] - 7s 219ms/step - loss: 0.4542 - acc: 0.0000e+00 - val_loss: 0.3841 - val_acc: 0.0000e+00
Epoch 5/20
30/30 [==============================] - 6s 210ms/step - loss: 0.4537 - acc: 0.0000e+00 - val_loss: 0.4471 - val_acc: 0.0000e+00
Epoch 6/20
30/30 [==============================] - 6s 204ms/step - loss: 0.4368 - acc: 0.0000e+00 - val_loss: 1.3613 - val_acc: 0.0000e+00
Epoch 7/20
30/30 [======================